In [10]:
import os, sys

from keras.models import Model
#from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
import string
#import matplotlib.pyplo

In [6]:
translation_file = open(r'E:\Pycharm_Projects\tf_prac\Encoder,Decoder,Seq2Seq\eng-spa\spa.txt',"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[1000:20000]
#print(pairs)

In [7]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [8]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [11]:
# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
spanish_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
spa_text_tokenized, spa_text_tokenizer = tokenize(spanish_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length spanish sentence: {}'.format(len(max(spa_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
spanish_vocab = len(spa_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Spanish vocabulary is of {} unique words".format(spanish_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length spanish sentence: 9
Maximum length english sentence: 5
Spanish vocabulary is of 7225 unique words
English vocabulary is of 3725 unique words


In [12]:
max_spanish_len = int(len(max(spa_text_tokenized,key=len)))
print(max_spanish_len)
max_english_len = int(len(max(eng_text_tokenized,key=len)))
print(max_english_len)
spa_pad_sentence = pad_sequences(spa_text_tokenized, max_spanish_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
spa_pad_sentence = spa_pad_sentence.reshape(*spa_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

9
5


In [16]:
input_sequence = Input(shape=(max_spanish_len,))
embedding = Embedding(input_dim=spanish_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [17]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding_3 (Embedding)     (None, 9, 128)            924800    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 5, 64)            0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 5, 64)             33024     
                                                                 
 time_distributed (TimeDistr  (None, 5, 3725)          242125    
 ibuted)                                                     

In [20]:
model_results = enc_dec_model.fit(spa_pad_sentence, eng_pad_sentence, batch_size=30, epochs=200)

Epoch 1/200
634/634 [==============================] - 33s 51ms/step - loss: 2.4874 - accuracy: 0.5414
Epoch 2/200
634/634 [==============================] - 33s 52ms/step - loss: 2.3808 - accuracy: 0.5533
Epoch 3/200
634/634 [==============================] - 33s 53ms/step - loss: 2.2799 - accuracy: 0.5672
Epoch 4/200
634/634 [==============================] - 33s 53ms/step - loss: 2.1847 - accuracy: 0.5789
Epoch 5/200
634/634 [==============================] - 33s 53ms/step - loss: 2.0966 - accuracy: 0.5882
Epoch 6/200
634/634 [==============================] - 32s 51ms/step - loss: 2.0107 - accuracy: 0.5992
Epoch 7/200
634/634 [==============================] - 34s 54ms/step - loss: 1.9319 - accuracy: 0.6099
Epoch 8/200
634/634 [==============================] - 34s 54ms/step - loss: 1.8571 - accuracy: 0.6210
Epoch 9/200
634/634 [==============================] - 35s 55ms/step - loss: 1.7855 - accuracy: 0.6297
Epoch 10/200
634/634 [==============================] - 34s 54ms/step - l

634/634 [==============================] - 37s 58ms/step - loss: 0.2839 - accuracy: 0.9156
Epoch 80/200
634/634 [==============================] - 38s 60ms/step - loss: 0.2794 - accuracy: 0.9168
Epoch 81/200
634/634 [==============================] - 37s 59ms/step - loss: 0.2750 - accuracy: 0.9174
Epoch 82/200
634/634 [==============================] - 37s 58ms/step - loss: 0.2745 - accuracy: 0.9176
Epoch 83/200
634/634 [==============================] - 36s 57ms/step - loss: 0.2692 - accuracy: 0.9187
Epoch 84/200
634/634 [==============================] - 34s 54ms/step - loss: 0.2654 - accuracy: 0.9197
Epoch 85/200
634/634 [==============================] - 34s 54ms/step - loss: 0.2671 - accuracy: 0.9189
Epoch 86/200
634/634 [==============================] - 35s 55ms/step - loss: 0.2615 - accuracy: 0.9205
Epoch 87/200
634/634 [==============================] - 35s 56ms/step - loss: 0.2550 - accuracy: 0.9218
Epoch 88/200
634/634 [==============================] - 35s 56ms/step - loss:

634/634 [==============================] - 36s 57ms/step - loss: 0.1652 - accuracy: 0.9404
Epoch 158/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1655 - accuracy: 0.9414
Epoch 159/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1637 - accuracy: 0.9417
Epoch 160/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1642 - accuracy: 0.9411
Epoch 161/200
634/634 [==============================] - 35s 56ms/step - loss: 0.1634 - accuracy: 0.9412
Epoch 162/200
634/634 [==============================] - 36s 56ms/step - loss: 0.1628 - accuracy: 0.9415
Epoch 163/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1630 - accuracy: 0.9410
Epoch 164/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1618 - accuracy: 0.9410
Epoch 165/200
634/634 [==============================] - 36s 57ms/step - loss: 0.1612 - accuracy: 0.9419
Epoch 166/200
634/634 [==============================] - 36s 57ms/ste

In [22]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 18
print("The english sentence is: {}".format(english_sentences[index]))
print("The spanish sentence is: {}".format(spanish_sentences[index]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(spa_pad_sentence[index:index+1])[0], eng_text_tokenizer))

The english sentence is: stay here
The spanish sentence is: quedate acá
The predicted sentence is :
1/1 [==============================] - 0s 36ms/step
stay here <empty> <empty> <empty>
